Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [47]:
import os

os.chdir('/')
!rm -r /CycleGAN-VAE-NST/
!git clone https://github.com/davexhardware/CycleGAN-VAE-NST

Cloning into 'CycleGAN-VAE-NST'...
remote: Enumerating objects: 2598, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 2598 (delta 42), reused 49 (delta 18), pack-reused 2515 (from 1)
Receiving objects: 100% (2598/2598), 8.28 MiB | 17.16 MiB/s, done.
Resolving deltas: 100% (1616/1616), done.


In [1]:
import os
os.chdir('/workspace/CycleGAN-VAE-NST/')

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 101.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 64.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 KB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 KB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 KB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 KB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 11.8 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-no

# Datasets

Use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. For our experiment, we're going to extract the tensors of the images from zip files.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cd /workspace/CycleGAN-VAE-NST
!rm -r trainB
!rm -r trainA
!unzip -d ./ -o ../onepiece_pt.zip
!unzip -d ./ -o ../img_align_celeba_pt.zip

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
rm: cannot remove 'trainB': No such file or directory
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
rm: cannot remove 'trainA': No such file or directory
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
unzip:  cannot find or open ./onepiece_pt.zip, ./onepiece_pt.zip.zip or ./onepiece_pt.zip.ZIP.
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
unzip:  cannot find or open ./img_align_celeba_pt.zip, ./img_align_celeba_pt.zip.zip or ./img_align_celeba_pt.zip.ZIP.


In [12]:
import os
import random
def move_train_to_test(train_dir, test_dir, num_files):
  if os.path.exists(train_dir):
    images = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]
    random.shuffle(images)
    images_to_move = images[num_files:]
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    for image in [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]:
      if image not in images_to_move:
          os.rename(os.path.join(train_dir, image), os.path.join(test_dir, image))

!cd /workspace/CycleGAN-VAE-NST
!rm -r ./datasets/trainA
!cp -r ./img_align_celeba_pt/ ./datasets/trainA
## COMMENT OUT THIS BLOCK IF YOU WANT TO KEEP ALL IMAGES
path='/workspace/CycleGAN-VAE-NST/datasets/trainA'
pathdst='/workspace/CycleGAN-VAE-NST/datasets/testA'
trainA_size=1000
move_train_to_test(path, pathdst, trainA_size)

!rm -r ./datasets/trainB
!cp -r ./onepiece_pt/ ./datasets/trainB
path='/workspace/CycleGAN-VAE-NST/datasets/trainB'
pathdst='/workspace/CycleGAN-VAE-NST/datasets/testB'
trainB_size=600
move_train_to_test(path, pathdst, trainB_size)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
total 168
drwxr-xr-x 13 vdb vdb  4096 Mar  5 10:04 .
drwxr-x--- 10 vdb vdb  4096 Mar  5 10:03 ..
drwxr-xr-x  8 vdb vdb  4096 Mar  5 09:37 .git
-rw-r--r--  1 vdb vdb   883 Mar  5 09:37 .gitignore
drwxr-xr-x  3 vdb vdb  4096 Mar  5 10:00 .idea
drwxr-xr-x  6 vdb vdb  4096 Mar  5 09:54 .venv
-rw-r--r--  1 vdb vdb 56889 Mar  5 10:04 CycleGAN.ipynb
-rw-r--r--  1 vdb vdb  3565 Mar  5 09:37 LICENSE
-rw-r--r--  1 vdb vdb 16461 Mar  5 09:47 README.md
drwxr-xr-x  3 vdb vdb  4096 Mar  5 09:37 data
drwxr-xr-x  4 vdb vdb  4096 Mar  5 09:57 datasets
drwxr-xr-x  2 vdb vdb  4096 Mar  5 09:37 docs
-rw-r--r--  1 vdb vdb   247 Mar  5 09:37 environment.yml
drwxr-xr-x  2 vdb vdb  4096 Mar  5 09:37 imgs
drwxr-xr-x  2 vdb vdb  4096 Mar  5 09:37 models
drwxr-xr-x  2 vdb vdb  4096 Mar  5 09:37 options
-rw-r--r--  1 vdb vdb  7122 Mar  5 09:37 pix2pix.ipynb
-rw-r--r--  1 vdb vdb    79 Mar  5 09:37 requirements.txt
drwxr-xr-x  2 vdb vdb  409

In [13]:
!ls ./datasets/trainB  | wc -l
!ls ./datasets/trainA  | wc -l

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
7603
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
19073


# Training


Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use  `--batch_size` to change the batch size.

In [16]:

import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("cuDNN enabled:", torch.backends.cudnn.enabled)
print("cuDNN version:", torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else "Not available")
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

x = torch.randn(1, 1, 32, 32, device="cuda")  # Random tensor on GPU
conv = torch.nn.Conv2d(1, 1, kernel_size=3).cuda()  # Simple Conv2D layer
with torch.backends.cudnn.flags(enabled=True):
    output = conv(x)
print("cuDNN acceleration is working!" if torch.backends.cudnn.enabled else "cuDNN is not being used.")

PyTorch version: 2.6.0+cu124
CUDA available: True
cuDNN enabled: True
cuDNN version: 90100
CUDA Device Name: NVIDIA GeForce RTX 4070
cuDNN acceleration is working!


In [ ]:
os.chdir('/workspace/CycleGAN-VAE-NST/')
    !python train.py --dataroot ./datasets --name portraits2op --lambda_identity 0 --latent_dim 20 --ngf 32 --ndf 32 --norm batch --netG resnet_6blocks --lambda_B 8.0 --dataset_mode tensor --crop_size 128 --batch_size 1 --verbose --n_epochs 20 --n_epochs_decay 10 --display_id -1

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 128                           	[default: 256]
                 dataroot: ./datasets                    	[default: None]
             dataset_mode: tensor                        	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epo


Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/<MODELNAME>/latest_net_G_A.pth ./checkpoints/<MODELNAME>/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./checkpoints/<MODELNAME>/web/images/epoch100_fakeA.jpg')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./checkpoints/<MODELNAME>/web/images/epoch100_realA.jpg')
plt.imshow(img)